In [1]:
from tqdm.notebook import tqdm
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import tensorflow as tf
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.models import load_model, Model, Sequential
from keras.layers import Dense, Flatten, Conv2D, Dropout, MaxPool2D, BatchNormalization, Dropout, Layer, LSTM, Input

In [2]:
dhw_merge = pd.read_csv('dhw_merge.csv')
elec_merge  = pd.read_csv('elec_merge.csv')
n_elec_merge = pd.read_csv('n_elec_merge.csv')

dhw_merge = dhw_merge.drop(['YEAR'], axis=1)
elec_merge = elec_merge.drop(['YEAR'], axis=1)
n_elec_merge = n_elec_merge.drop(['YEAR'], axis=1)

##  For Scaling

In [3]:
dhw_X = dhw_merge.drop(['DHW'], axis=1)
dhw_Y = dhw_merge[['DHW']]

scaler_dhw = MinMaxScaler()
scaler_dhw.fit(dhw_X)
scaled_dhw_X = scaler_dhw.transform(dhw_X)

new_dhw_X = pd.DataFrame(scaled_dhw_X, index=dhw_X.index, columns=dhw_X.columns)
new_dhw = pd.concat([new_dhw_X, dhw_Y], axis=1)

In [4]:
elec_X = elec_merge.drop(['ELEC'], axis=1)
elec_Y = elec_merge[['ELEC']]

scaler_elec = MinMaxScaler()
scaler_elec.fit(elec_X)
scaled_elec_X = scaler_elec.transform(elec_X)

new_elec_X = pd.DataFrame(scaled_elec_X, index=elec_X.index, columns=elec_X.columns)
new_elec = pd.concat([new_elec_X, elec_Y], axis=1)

In [5]:
n_elec_X = n_elec_merge.drop(['n_elec'], axis=1)
n_elec_Y = n_elec_merge[['n_elec']]

scaler_n_elec = MinMaxScaler()
scaler_n_elec.fit(n_elec_X)
scaled_n_elec_X = scaler_n_elec.transform(n_elec_X)

new_n_elec_X = pd.DataFrame(scaled_n_elec_X, index=n_elec_X.index, columns=n_elec_X.columns)
new_n_elec = pd.concat([new_n_elec_X, n_elec_Y], axis=1)

In [6]:
input_dhw = new_dhw.iloc[:-1,]
target_dhw = new_dhw[['DHW']].iloc[1:]

trainX_dhw, testX_dhw, trainY_dhw, testY_dhw = train_test_split(input_dhw,target_dhw,test_size=0.3,shuffle=False,random_state=0)

In [7]:
input_elec = new_elec.iloc[:-1,]
target_elec = new_elec[['ELEC']].iloc[1:]

trainX_elec, testX_elec, trainY_elec, testY_elec = train_test_split(input_elec,target_elec,test_size=0.3,shuffle=False,random_state=0)

In [8]:
input = new_n_elec.iloc[:-1,]
target = new_n_elec[['n_elec']].iloc[1:]

trainX_n_elec, testX_n_elec, trainY_n_elec, testY_n_elec = train_test_split(input,target,test_size=0.3,shuffle=False,random_state=0)

In [9]:
def buildDataSet(input, target, seqLength):
    xdata = []
    ydata = []
    for i in range(len(input) - seqLength):
        tx = input.iloc[i:i+seqLength]
        ty = target.iloc[i+seqLength-1]
        xdata.append(tx)
        ydata.append(ty)
    return np.array(xdata), np.array(ydata)

In [10]:
trainx_dhw, trainy_dhw = buildDataSet(trainX_dhw, trainY_dhw, 3)
testx_dhw, testy_dhw = buildDataSet(testX_dhw, testY_dhw, 3)
trainx_elec, trainy_elec = buildDataSet(trainX_elec, trainY_elec, 3)
testx_elec, testy_elec = buildDataSet(testX_elec, testY_elec, 3)
trainx_n_elec, trainy_n_elec = buildDataSet(trainX_n_elec, trainY_n_elec, 3)
testx_n_elec, testy_n_elec = buildDataSet(testX_n_elec, testY_n_elec, 3)

## Reverse MinMaxScaler

In [11]:
# reverse_train_x = scaler_dhw_x_train.inverse_transform(scaled_dhw_train_x)
# reverse_train_y = scaler_dhw_y_train.inverse_transform(scaled_dhw_train_y)
# reverse_test_x = scaler_dhw_x_train.inverse_transform(scaled_dhw_test_x)
# reverse_test_y = scaler_dhw_y_train.inverse_transform(scaled_dhw_test_y)

# rev_dhw_train_x = pd.DataFrame(reverse_train_x, index=dhw_x_train.index, columns=dhw_x_train.columns)
# rev_dhw_train_y = pd.DataFrame(reverse_train_y, index=dhw_y_train.index, columns=dhw_y_train.columns)

# rev_dhw_test_x = pd.DataFrame(reverse_test_x, index=dhw_x_test.index, columns=dhw_x_test.columns)
# rev_dhw_test_y = pd.DataFrame(reverse_test_y, index=dhw_y_test.index, columns=dhw_y_test.columns)

# rev_dhw_trainSet = pd.concat([rev_dhw_train_x, rev_dhw_train_y], axis=1)
# rev_dhw_testSet = pd.concat([rev_dhw_test_x, rev_dhw_test_y], axis=1)

In [62]:
class DANN(Model):
    def __init__(self):
        super(DANN, self).__init__()

        # Feature Extractor
        self.feature_extractor = Sequential([
           LSTM(64, activation='swish', return_sequences=True),
           LSTM(64, activation='swish', return_sequences=False)
        ])
        
        # Label Predictor
        self.label_predictor = Sequential([
            Dense(32, activation='relu'),
            Dense(1) # 에너지 소비량을 예측하기 위한 레이어
        ])
        
        # Domain Predictor
        self.domain_predictor = Sequential([
            Dense(32, activation='relu'),
            Dense(1, activation='softmax') # 도메인 분류를 위한 레이어 (0:Source, 1:Target)
        ])
        
        self.predict_label = Sequential([
			self.feature_extractor,
			self.label_predictor
		])

		self.classify_domain = Sequential([
			self.feature_extractor,
			self.domain_classifier
		])
  
    @tf.function
	def train(self, x_class, y_class, x_domain):
		
		domain_labels = np.concatenate([np.zeros(len(x_class)), np.ones(len(x_domain))])
		
		x_both = tf.concat([x_class, x_domain], axis = 0)
		
		with tf.GradientTape() as tape:
			y_class_pred = self.predict_label(x_class, training=True)
			lp_loss = self.loss(y_class, y_class_pred)   
		lp_grad = tape.gradient(lp_loss, self.predict_label.trainable_variables)

		with tf.GradientTape(persistent = True) as tape:
			y_domain_pred = self.classify_domain(x_both, training=True)
			dc_loss = self.loss(domain_labels, y_domain_pred) 
		fe_grad = tape.gradient(dc_loss, self.feature_extractor.trainable_variables)
		dc_grad = tape.gradient(dc_loss, self.domain_classifier.trainable_variables)
		del tape

		self.lp_optimizer.apply_gradients(zip(lp_grad, self.predict_label.trainable_variables))
		# self.dc_optimizer.apply_gradients(zip(dc_grad, self.classify_domain.trainable_variables))
		self.dc_optimizer.apply_gradients(zip(dc_grad, self.domain_classifier.trainable_variables))
		
		self.fe_optimizer.apply_gradients(zip(fe_grad, self.feature_extractor.trainable_variables))

		self.train_lp_loss(lp_loss)
		self.train_lp_accuracy(y_class, y_class_pred)
		
		self.train_dc_loss(dc_loss)
		self.train_dc_accuracy(domain_labels, y_domain_pred)

		return

    def call(self, inputs, train=False, lamda=1.0):
        #model_input = self.model_input(inputs)
        lstm_features1 = self.lstm_layer1(inputs)
        lstm_features2 = self.lstm_layer2(lstm_features1)
        #features = self.feature_extractor(lstm_features2)
        power_consumption_prediction = self.label_predictor(lstm_features2)
        
        if train:
            reversed_features = GradientReversalLayer(lamda)(lstm_features2)
            domain_prediction = self.domain_predictor(reversed_features)
            return power_consumption_prediction, domain_prediction
        
        else:
            return power_consumption_prediction
################################################        
@tf.custom_gradient
def GradientReversalOperator(x):
	def grad(dy):
		return -1 * dy
	return x, grad

class GradientReversalLayer(tf.keras.layers.Layer):
	def __init__(self):
		super(GradientReversalLayer, self).__init__()
		
	def call(self, inputs):
		return GradientReversalOperator(inputs)

In [63]:
dann_model = DANN()

#predictions = dann_model(trainx_n_elec)

In [64]:
#dann_model.compile(optimizer='adam', loss='mae')

In [65]:
#dann_model.fit(trainx_n_elec, trainy_n_elec, epochs=100, batch_size=1024, train=True)

In [66]:
# 옵티마이저 및 손실 함수 설정
optimizer = tf.keras.optimizers.Adam()
loss_object = tf.keras.losses.MeanSquaredError()

# 학습 설정
epochs = 100
lamda = 1.0
batch_size = 16

# 학습 루프
for epoch in range(epochs):
    n_elec_loss = 0.0
    elec_loss = 0.0
    
    # 배치별 학습
    for i in range(0, len(trainx_n_elec), batch_size):
        batch_n_elec = trainx_n_elec[i:i+batch_size]
        batch_n_elec_target = trainy_n_elec[i:i+batch_size]
        batch_elec = trainx_elec[i:i+batch_size]
        batch_elec_target = trainy_elec[i:i+batch_size]
        
        with tf.GradientTape(persistent=True) as tape:
            # 소스 도메인 데이터에 대한 레이블 예측 및 도메인 분류
            n_elec_power_prediction, n_elec_domain_prediction = dann_model(batch_n_elec, train=True, lamda=lamda)
            n_elec_power_loss = loss_object(batch_n_elec_target, n_elec_power_prediction)
            n_elec_domain_loss = loss_object(tf.zeros_like(n_elec_domain_prediction), n_elec_domain_prediction)
            total_n_elec_loss = n_elec_power_loss + n_elec_domain_loss
            
            # 타겟 도메인 데이터에 대한 레이블 예측 및 도메인 분류
            elec_power_prediction, elec_domain_prediction = dann_model(batch_elec, train=True, lamda=lamda)
            elec_power_loss = loss_object(batch_elec_target, elec_power_prediction)
            elec_domain_loss = loss_object(tf.ones_like(elec_domain_prediction), elec_domain_prediction)
            total_elec_loss = elec_power_loss + elec_domain_loss
        
        # 소스 도메인과 타겟 도메인의 합친 손실을 최소화하는 방향으로 가중치 업데이트
        total_loss = total_n_elec_loss + total_elec

ValueError: Exception encountered when calling layer "gradient_reversal_layer_6" (type GradientReversalLayer).

Attempt to convert a value (<function identity at 0x00000239BFA00A60>) with an unsupported type (<class 'function'>) to a Tensor.

Call arguments received by layer "gradient_reversal_layer_6" (type GradientReversalLayer):
  • inputs=tf.Tensor(shape=(16, 64), dtype=float32)

In [ ]:
# 학습 이전의 모델 에측 결과 시각화
plt.figure(figsize=(12,6))
plt.scatter(range(num_sampels), city_a_data, label='City A Ground Truth')
plt.sactter(range(num_samples), dann_model(city_a_data), label='City A predction (Before Adaptation)')
plt.scatter(range(num_samples), city_b_data, label='City B Ground Truth')
plt.sactter(range(num_samples), dann_model(city_b_data), label='City B prediction (Before Adaptation)')
plt.xlabel('Sample Index')
plt.ylabel('Power Consumption')
plt.legend()
plt.title('Prediction Before Adaptation')
plt.show()

In [ ]:
# 학습 후의 모델 예측 결과 시각화
plt.figure(figsize=(12,6))
plt.scatter(range(num_samples), city_a_data, label='City A Ground Truth')
plt.scatter(range(num_samples), dann_model(city_a_data), label='City A predcition (After Adaptation)')
plt.scatter(range(num_samples), city_b_data, label='City B Ground Truth')
plt.sactter(range(num_samples), dann_model(city_b_data), label='City B prediction (After Adaptation)')
plt.xlabel('Sample Index')
plt.ylabel('Power Consumption')
plt.legend()
plt.title('Prediction After Adaptation')
plt.show()

In [ ]:
# from sklearn.metrics import mean_absolute_error
# print('MAE : ', round(mean_absolute_error(real, pred),4)) 

# from sklearn.metrics import mean_squared_error
# print('MSE : ', round(mean_squared_error(real, pred),4))

# from sklearn.metrics import r2_score
# print('R2 : ', round(r2_score(real, pred),4))